In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import aseg_gdf2
from shapely.geometry import Polygon

In [2]:
infile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\inversion_ready\DR_inversion_ready"

#infile = r"C:\Users\PCUser\Desktop\EK_data\AEM\inversion_ready_data\Keep_inversion_ready.dat"
dat = aseg_gdf2.read(infile)

In [3]:
df_AEM = dat.df()

# Convert to geopandas data frame
gdf_AEM = gpd.GeoDataFrame(df_AEM, 
                           geometry=gpd.points_from_xy(df_AEM['easting '],
                                                       df_AEM['northing']))

In [5]:
# Bring in the polygon data for subseting

infile = r"C:\Users\PCUser\Desktop\NSC_data\data\vector\DalyRiver\DalyGeology_August2018.shp"



gdf = gpd.read_file(infile)

# Convert to easting northing
gdf = gdf.to_crs({'init': 'epsg:28352'})

poly = gdf['geometry'].iloc[8]


In [6]:
newPolygon = Polygon(poly.exterior)

In [7]:
gdf_AEM['inside_poly'] = gdf_AEM.within(newPolygon)

In [47]:
gdf_AEM[gdf_AEM['fiducial '] == 1582705.5]

,project,date,flight,line,fiducial,easting,northing,height,elevation,GPS_Alt,...,RUNC_HM_Z[15],RUNC_HM_Z[16],RUNC_HM_Z[17],RUNC_HM_Z[18],RUNC_HM_Z[19],RUNC_HM_Z[20],RUNC_HM_Z[21],RUNC_HM_Z[22],geometry,inside_poly
166984,1304,20170819,20170819.02,109701,1582705.5,866248.06,8394875.0,50.4,154.2,251.68,...,0.008284,0.004474,0.002119,0.000977,0.000576,0.000456,0.000396,0.000358,POINT (866248.060 8394875.000),False


In [8]:
# Now we get the indices of points that are inside the poly
#gdf_subset = gdf_AEM.copy()
gdf_subset = gdf_AEM[gdf_AEM['inside_poly'] == True]

In [11]:
# Lets just keep certain lines

lines = [110101, 110201, 110103, 109301, 109502, 109503, 101501, 101502, 104501,
         110002, 108301, 104801, 103502]

new_lines =  [106501, 106601, 106701, 106801, 106901, 107001, 107101,
             107201, 107301, 107401, 107501, 107601, 107701, 107801,
             107901, 108801, 108101, 108201, 108301, 108401, 108501,
             108601, 108701, 108801, 108901, 108001, 109101]

#lines = pd.read_csv(r"C:\temp\EK_3d_model_lines.csv").values.flatten()

gdf_subset = gdf_AEM[gdf_AEM['line '].isin(new_lines)]

In [12]:
# Now we write out our data into a text file using the indices of the dataframe
infile =  r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\inversion_ready\DR_inversion_ready.dat"
outfile =  r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\inversion_ready\DR_inversion_ready_NS_subset_3.dat"

inf = open(infile)
lines = inf.readlines()

with open(outfile, 'w') as outf:
    for item in gdf_subset.index.tolist():
        outf.write(lines[item])
        